In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle

windows = False

## Baseline Conditions

#### For Attention List:
1. Need to go through and check the units work properly. Note that the values didnt change when I removed units so seems okay
2. Check units on flows
3. Check units on Vt
4. Add units to to all columns and decide upon my convention for this
5. Check units and speak to Payne about the wall thickness calcs
6. Is velocity redundant?

#### Set Constants and paramaters

In [2]:
df = None

#input params, _o stands for original
H = 0.42 #no units,ratio #Hematocrit assumed to be constant
# M_o = 30* 10e-9 #mol_ O2/(mL/s) #Taken from table 2 from Wiley Payne paper
cHb = 0.2 #mL_O2/m #Taken from table 2 from Wiley Payne paper
paO2_bar_t = 15 #mmHG #Taken from table 2 from Wiley Payne paper
K_o = 5e-8 #µL/(mm*s*mmHg) #payne paper and boas et al

##reset df
df = None

#Input data from paper
df = pd.DataFrame({'Name': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'C', 'V6', 'V5', 'V4', 'V3', 'V2', 'V1'], 
                   'Number': [1, 2, 4, 8, 16, 32, 64, 32, 16, 8, 4, 2, 1],
                   'Diameter(µm)': [23.97, 19.17, 15.28, 12.08, 9.46, 7.32, 8, 11.51, 14.53, 17.79, 21.45, 25.70, 30.77],
                   'Length(µm)': [1267.6, 930.3, 543.6, 302.3, 161.2, 154.7, 243.9, 473.9, 272.3, 426.6, 632.5, 844.2, 936.3],
                   'Viscocity(mPAs)': [1.59, 1.50, 1.42, 1.34, 1.28, 1.23, 1.24, 1.33, 1.40, 1.48, 1.55, 1.62, 1.7],
                   'Pressure Drop(mmHg)': [6.93, 5.87, 4.02, 2.70, 1.82, 2.35, 2.62, 1.27, 0.61, 0.89, 1.31, 1.78, 2.01],
                   'Saturation in': [0.94, 0.93, 0.92, 0.89, 0.84, 0.765, 0.665, 0.61, 0.5975, 0.5875, 0.5825, 0.5775, 0.5725],
                   'Saturation out': [0.93, 0.92, 0.89, 0.84, 0.765, 0.665, 0.61, 0.5975, 0.5875, 0.5825, 0.5775, 0.5725, 0.5675],
})

#convert param to my SU
M_constant = 8.20e-4 # cm3_O2/(cm3*s) , still unsure about the exact conversion so will just input in this section
K = K_o * (10**6) # µm2/(s*mmHg)
df['Viscocity(mmHg*s)'] = df['Viscocity(mPAs)'] / (133.3e3)
df = df.drop(['Viscocity(mPAs)'],axis=1)

#### Calculate Flows and Vt

In [3]:
#Calculate resistances of each vessel
df['Resistance for U'] = 32 * df['Viscocity(mmHg*s)'] * df['Length(µm)'].div(df['Diameter(µm)']**2)
df['Resistance for Q'] = 128 * df['Viscocity(mmHg*s)'] * df['Length(µm)'].div(math.pi*(df['Diameter(µm)']**4))

#Alter resistances for C
#These are aross each pressure drop (explains the doubled resistance for the capillaries as in paralell so have half the flow each (flow is shared))
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U']*2
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q']*2

#Calculate flow and velocity in a single vessel
df['U in single(µm/s)'] = df['Pressure Drop(mmHg)'].div(df['Resistance for U'])
df['Q in single(µm3/s)'] = df['Pressure Drop(mmHg)'].div(df['Resistance for Q'])

#Calculate dS and Average Saturation
df['dS'] = df['Saturation in'] - df['Saturation out']
df['Saturation ave'] = df['Saturation in'] - 0.5 * df['dS']

#### Calculate partial pressure in blood
S and partial O2 relationship given by servinghaus 1979  
Checked partial pressure against typical values online and it seems bang on.

In [4]:
def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 1e-13 and all_roots[i].imag >= -1e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols
 
for i in range(len(df['Saturation ave'])):
    root_temp = None
    sols = 0
    root_temp = np.roots([1,0,150,23400*df.loc[i,'Saturation ave']/(df.loc[i,'Saturation ave']-1)]) #gives mmHg
    sols = check_imag_roots_real(root_temp) 
    #sols = sols*133.322 ###Remove this step if want in mmHg
    df.loc[i,'partial pressure blood(mmHg)'] = sols

#### Calculate wall thickness
Will take my vessel average partial pressure to be the 0.5(pin + pout)  
All units were in mm so i played around with this equation from the paper  
Equations are taken from Payne paper

In [5]:
#all units were in mm so i played around with this equation from the paper
df['X-Area wall(???)'] = np.pi * ( 0.16*(df['Diameter(µm)'])**2 + 1.4*(df['Diameter(µm)']) + 14 )
df['wall thickness(µm)'] = ( -(df['Diameter(µm)']) + np.sqrt((df['Diameter(µm)'])**2 + (4*(df['X-Area wall(???)'])/np.pi)) ) / 2

#### Calculate partial pressure in tissue
Equations can be found on the week 1 or 2 notes from class. From steady state version of something on the paper

In [6]:
M_constant = 8.20e-4 # cm3_O2/(cm3*s)
pt = {}
M = {}
Vt = {}
M[0] = M_constant
hill_constant = 2 # mmHG

length = 1001
for i in range(length):
    #Calculate Vt
    Vt[i] = df['Q in single(µm3/s)']*cHb*H*df['dS']/M[i]
    pt[i] = df['partial pressure blood(mmHg)'] - M[i] * Vt[i] * df['wall thickness(µm)'] / (2 * np.pi * K * (df['Diameter(µm)']/2) * df['Length(µm)'])
    M[i+1] = M_constant*pt[i]/(pt[i]+hill_constant)
#     if i != 0:
#         temp = pt[i].values-pt[i-1].values
#         if temp.value < 0.01:
#             print(pt[i])
#             break

df['tissue partials(mmHg)'] = pt[1000]
df['Vt(µm3)'] = Vt[1000]
# df['tissue partials(mmHg)'] = df['partial pressure blood(mmHg)'] - M * df['Vt(µm3)'] * df['wall thickness(µm)'] / (2 * np.pi * K * (df['Diameter(µm)']/2) * df['Length(µm)'])

#### Pull out the baseline conditions

In [7]:
df_base = df
df_base

,Name,Number,Diameter(µm),Length(µm),Pressure Drop(mmHg),Saturation in,Saturation out,Viscocity(mmHg*s),Resistance for U,Resistance for Q,U in single(µm/s),Q in single(µm3/s),dS,Saturation ave,partial pressure blood(mmHg),X-Area wall(???),wall thickness(µm),tissue partials(mmHg),Vt(µm3)
0,A1,1,23.97,1267.6,6.93,0.9400,0.9300,0.000012,0.000842,1.866110e-06,8229.436888,3.713608e+06,0.0100,0.93500,68.843131,438.213672,4.841407,65.678837,3.920026e+06
1,A2,2,19.17,930.3,5.87,0.9300,0.9200,0.000011,0.000912,3.158317e-06,6439.441736,1.858585e+06,0.0100,0.92500,65.327798,313.016429,4.253658,62.957193,1.964399e+06
2,A3,4,15.28,543.6,4.02,0.9200,0.8900,0.000011,0.000794,4.328170e-06,5065.066367,9.287990e+05,0.0300,0.90500,59.809062,228.546291,3.810683,52.973057,2.962124e+06
3,A4,8,12.08,302.3,2.70,0.8900,0.8400,0.000010,0.000666,5.814403e-06,4051.677394,4.643641e+05,0.0500,0.86500,52.183980,170.463521,3.485850,40.331980,2.496394e+06
4,A5,16,9.46,161.2,1.82,0.8400,0.7650,0.000010,0.000553,7.874816e-06,3288.204541,2.311165e+05,0.0750,0.80250,44.546726,130.572935,3.265971,24.694563,1.919460e+06
5,A6,32,7.32,154.7,2.35,0.7650,0.6650,0.000009,0.000852,2.025728e-05,2756.606465,1.160077e+05,0.1000,0.71500,37.579012,103.110791,3.138293,20.386542,1.304955e+06
6,C,64,8.00,243.9,2.62,0.6650,0.6100,0.000009,0.002269,4.513708e-05,1154.776548,5.804540e+04,0.0550,0.63750,33.077299,111.338044,3.172168,30.301779,3.486216e+05
7,V6,32,11.51,473.9,1.27,0.6100,0.5975,0.000010,0.001142,1.097658e-05,1111.977986,1.157009e+05,0.0125,0.60375,31.395007,161.197683,3.433626,30.908155,1.577403e+05
8,V5,16,14.53,272.3,0.61,0.5975,0.5875,0.000011,0.000433,2.614226e-06,1407.231531,2.333386e+05,0.0100,0.59250,30.861700,214.009514,3.730525,29.685171,2.551342e+05
9,V4,8,17.79,426.6,0.89,0.5875,0.5825,0.000011,0.000479,1.926676e-06,1858.401329,4.619354e+05,0.0050,0.58500,30.512889,281.309064,4.092089,29.846912,2.524554e+05


late check that i calculate r right

In [12]:
R = df_base['Resistance for Q']

In [15]:
R_tot = R[0] + R[1]/2+ R[2]/2**2 + R[3]/2**3 + R[4]/2**4 + R[5]/2**5 + R[6]/2**6 + R[7]/2**5 + R[8]/2**4 + R[9]/2**3 + R[10]/2**2 + R[11]/2**1 + R[12]
print(R_tot)

9.207545330665768e-06


In [97]:
if windows == True:
    df_base.to_csv(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\baselines_new.csv") # FOR WINDOWS
else:
    df_base.to_csv(r"/Users/Debs/OneDrive - Nexus365/4YP/python/_storage/baselines_new.csv") # FOR MAC
    
#### CHANGE TO BASELINES WITHIN FOLDER. this means i dont need to worry about changing baselines for later calcls unless i absolutely want to

In [13]:
# df_base['pt_by_Vt'] = df_base['tissue partials(mmHg)'] * df_base'Vt(µm3)']
pt_averaged = (df_base['tissue partials(mmHg)'] * df_base['Vt(µm3)']).sum()/13

In [14]:
pt_averaged #checked against other model and seems right.

64942005.66676894

In [19]:
save = True
if save == True:
    if windows == True:
        pickle_out = open(r"C:\Users\timst\Onedrive - Nexus365\4YP\python\_storage\baselines\baselines_pt_averaged.pickle",'wb') #wb for writing binary and rb for reading binary
        pickle.dump(pt_averaged, pickle_out)
        pickle_out.close()
    else:
        pickle_out = open(r"/Users/Debs/OneDrive - Nexus365/4YP/python/_storage/baselines/baselines_pt_averaged.pickle",'wb')
        pickle.dump(pt_averaged, pickle_out)
        pickle_out.close()
#### CHANGE TO BASELINES WITHIN FOLDER. this means i dont need to worry about changing baselines for later calcls unless i absolutely want to